In [1]:
from pdf2image import convert_from_path, convert_from_bytes
import torch
from functools import partial
from PIL import Image
from transformers import Pix2StructForConditionalGeneration as psg
from transformers import Pix2StructProcessor as psp

/home/farouk/BEiT_CV_parsing/pix2struct/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = psg.from_pretrained("google/pix2struct-docvqa-large").to(DEVICE)
processor = psp.from_pretrained("google/pix2struct-docvqa-large")

In [3]:
def generate(model, processor, img, questions):
  inputs = processor(images=[img for _ in range(len(questions))], 
           text=questions, return_tensors="pt").to(DEVICE)
  predictions = model.generate(**inputs, max_new_tokens=256)
  return zip(questions, processor.batch_decode(predictions, skip_special_tokens=True))

def convert_pdf_to_image(filename, page_no):
    return convert_from_path(filename)[page_no-1]

In [4]:
FILENAME = "/home/farouk/BEiT_CV_parsing/cvparsing_v1_203/train/Aaheli Bhattacharjee (CN092983)/Aaheli Bhattacharjee (CN092983).pdf"
PAGE_NO = 1

In [5]:
questions = ["Outside of your professional career, what are your other interests and hobbies?"]

In [16]:
questions = ["how much time did he work as Consultant at Halliburton?"]

In [7]:
image = convert_pdf_to_image(FILENAME, PAGE_NO)
print("pdf to image conversion complete.")
generator = partial(generate, model, processor)
completions = generator(image, questions)
for completion in completions:
    print(f"{completion}") 

pdf to image conversion complete.
('Outside of your professional career, what are your other interests and hobbies?', 'Aaheli Bhattacharjee')


In [10]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = psg.from_pretrained("google/pix2struct-chartqa-base").to(DEVICE)
processor = psp.from_pretrained("google/pix2struct-chartqa-base")